# import

In [1]:
import pandas as pd
import numpy as np
from model import GoalProbEvaluator
import yfinance as yf

from utils import get_eom_dates

# All Weather Portfolio

In [2]:
# ALL WEATHER PORTFOLIO (MONTHLY REBALANCED)
df = yf.download(['SPY', 'TLT', 'IEF', 'GLD', 'DBC'])['Adj Close'].dropna()
rtns = np.log(df / df.shift(1)).fillna(0)

rebal_dates = get_eom_dates(rtns, start='2008-1-1', end='2024-6-30')
weights = pd.DataFrame(np.array([[0.3, 0.4, 0.15, 0.075, 0.075]] * len(rebal_dates)), columns=rtns.columns,
                       index=rebal_dates)

[*********************100%***********************]  5 of 5 completed


# Config

- initial_W : Initial capital


- T : Number of rebalancing times


- h : 1 if yearly, 1/4 if quarterly, 1/12 if monthly


- rho : The range of Wealth to be generated
    - The higher the rho value, the lower the probability of generating such Wealth (e.g., 3sigma, 4sigma, ...)
    
    
- bankrupt : The level of Wealth considered to be practical bankruptcy


- imax : The number of Wealth to be generated at each time point (T=t)
    -The more there are, the longer the optimization time
    
    
- lb:  Lower bound for each asset


- ub:  Upper bound for each asset

In [19]:
# INITIALIZE CONFIG
config = {
    'initial_W': 100,
    'T': len(weights.index),
    'bankrupt': 5,  # bankruptcy level
    'h': 1 / 12,  # monthly / if you want to calculate quarterly, set it to 1 / 4
    'imax': 1000,
    'rho': 2.5,   # a scale parameter
    'lb': [0.25, 0.35, 0.10, 0.025, 0.025],  # A little buffer
    'ub': [0.35, 0.45, 0.20, 0.125, 0.125]
}

- er_method: method for estimating expected returns of assets
- cov_method: method for estimating Covariances
- window: window size

In [20]:
est_config = {
    'er_method': 'bayes_stein',
    'cov_method': 'epo',
    'window': 252 * 30
}

# CashFlows

In [21]:
# Include the scheduled cash flows
cashflows = np.ones((config["T"],)) * 0

# Run

In [22]:
model = GoalProbEvaluator(rtns, weights, config, est_config)
model.add_cashflow(cashflows)

model.fit(fit_dist=True, display_result=True)

********************
WEALTH STATES GENERATED
********************
********************
PATHS GENERATED
********************
Expected Wealth: 100.0 -> 101.0932 -> 102.3122 -> 103.4061 -> 104.3896 -> 105.3946 -> 106.5619 -> 107.454 -> 108.1408 -> 108.6392 -> 108.5655 -> 108.6174 -> 108.8226 -> 109.0236 -> 109.249 -> 109.5365 -> 109.8554 -> 110.3472 -> 110.8469 -> 111.352 -> 111.9267 -> 112.5178 -> 113.2239 -> 114.0549 -> 114.7759 -> 115.4178 -> 116.0851 -> 116.76 -> 117.5481 -> 118.2965 -> 119.045 -> 119.7873 -> 120.6073 -> 121.5532 -> 122.5302 -> 123.496 -> 124.5001 -> 125.4666 -> 126.5204 -> 127.5794 -> 128.7583 -> 129.9077 -> 130.9869 -> 132.1573 -> 133.4419 -> 134.5397 -> 135.7139 -> 136.8395 -> 137.8834 -> 139.0467 -> 140.2751 -> 141.3964 -> 142.5244 -> 143.5372 -> 144.548 -> 145.6617 -> 146.8042 -> 147.9902 -> 149.1004 -> 150.229 -> 151.3052 -> 152.4019 -> 153.4532 -> 154.5275 -> 155.5675 -> 156.5267 -> 157.3469 -> 158.2511 -> 159.1998 -> 160.1033 -> 161.0266 -> 161.883 -> 162.7291

# Evaluate

- evaluate() arguments
    - goals: scalar if single goal, iterable objects(e.g. tuple, list, etc.) if multi goals
    - init_wealth: wealth value at start_idx (efault value is None, which means config['initial_W'])
    - start_idx: index value of the datetime at which the probability is calculated (default value is 0)
    - end_idx: index value of the datetime at which the goals are expected to achieve ((default value is -1)

## Case 1

- An investor wants to know the probability that the wealth, which was 100(default) in the very beginning, will be 300 at the final datetime

In [26]:
model.evaluate(goals=300)

,start date,end date,wealth(start),wealth(end),Probability
0,2008-01-31,END,100,300,0.4773


- The probability is evaluated to 47.73%

## Case 2

- An investor wants to know the probability that the wealth, which was 200 in January 2020, will be 300 at the final datetime

In [27]:
start_idx = rebal_dates.get_loc(rebal_dates[rebal_dates > '2020-1'][0])
model.evaluate(goals=300, init_wealth=200, start_idx=start_idx, end_idx=-1, decimal=4)

,start date,end date,wealth(start),wealth(end),Probability
0,2020-01-31,END,200,300,0.4122


- The probability is evaluated to 41.22%

## Case 3

- An investor wants to know the probability that the wealth, which was 200 in January 2020, will be 220 or 250 in December 2021

In [29]:
start_idx = rebal_dates.get_loc(rebal_dates[rebal_dates > '2020-1'][0])
end_idx = rebal_dates.get_loc(rebal_dates[rebal_dates > '2021-12'][0])
model.evaluate(goals=[220, 250], init_wealth=200, start_idx=start_idx, end_idx=end_idx, decimal=4)

,start date,end date,wealth(start),wealth(end),Probability
0,2020-01-31,2021-12-31,200,220,0.5606
1,2020-01-31,2021-12-31,200,250,0.3607
